In [ ]:
%pip install torch torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# --- AYARLAR ---
# Buraya doğru klasör yolunu yapıştırıyoruz (içinde Empty/Occupied olan)
VERI_YOLU = r"D:\2012-09-12"
# ---------------

# 1. Veri Hazırlığı (Hocanımızın İstediği %80-%20 Ayrımı)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

try:
    # 1. Tüm veriyi yükle
    full_dataset = datasets.ImageFolder(root=VERI_YOLU, transform=transform)
    total_count = len(full_dataset)
    
    # 2. %80 ve %20 sayılarını hesapla
    train_count = int(0.8 * total_count)
    test_count = total_count - train_count
    
    # 3. Veriyi rastgele böl
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_count, test_count])
    
    # 4. Yükleyicileri (Loader) hazırla
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    print(f"Veri Hazır!")
    # --- SINIFLARI YAZDIR ---
    print(f"Bulunan Sınıflar: {full_dataset.classes}") 
    # ------------------------
    print(f"Toplam Resim: {total_count}")
    print(f"Eğitim İçin (%80): {len(train_dataset)} adet")
    print(f"Test İçin (%20): {len(test_dataset)} adet")
    
except Exception as e:
    print("HATA: Veri yolu yanlış veya klasör boş!")
    print(e)

# 2. Model Mimarisi (Muhammet Emin - Wide CNN)
class WideCNN(nn.Module):
    def __init__(self):
        super(WideCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        
        # FARK BURADA: kernel_size=5 (Daha geniş bir alana bakıyor)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2) 
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        return x

# 3. Eğitim Döngüsü
if 'train_loader' in locals():
    model = WideCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print("\n--- EĞİTİM BAŞLIYOR (%80 Veri ile) ---")
    for epoch in range(3): # 3 Epoch
        model.train()
        running_loss = 0.0
        
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        print(f"Epoch {epoch+1} tamamlandı. Ortalama Kayıp: {running_loss/len(train_loader):.4f}")

    print("Eğitim Bitti.")

    # 4. TEST AŞAMASI
    print("\n--- TEST BAŞLIYOR (%20 Görmediği Veri ile) ---")
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"TEST SONUCU (Accuracy): %{test_accuracy:.2f}")
    print("------------------------------------------------")

Veri Hazır!
Bulunan Sınıflar: ['Empty', 'Occupied']
Toplam Resim: 8999
Eğitim İçin (%80): 7199 adet
Test İçin (%20): 1800 adet

--- EĞİTİM BAŞLIYOR (%80 Veri ile) ---
Epoch 1 tamamlandı. Ortalama Kayıp: 0.0598
Epoch 2 tamamlandı. Ortalama Kayıp: 0.0340
Epoch 3 tamamlandı. Ortalama Kayıp: 0.0050
Eğitim Bitti.

--- TEST BAŞLIYOR (%20 Görmediği Veri ile) ---
TEST SONUCU (Accuracy): %99.78
------------------------------------------------
